In [3]:
import pandas as pd
import numpy as np
import get_secrets
from alpaca.trading.client import TradingClient
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest
from alpaca.trading.requests import MarketOrderRequest, GetAssetsRequest
from alpaca.trading.enums import OrderSide, TimeInForce, AssetClass


In [ ]:
secrets = get_secrets.get_secrets('.secrets')

KEY = secrets[8]
SECRET_KEY = secrets[9]

data_client = StockHistoricalDataClient(KEY, SECRET_KEY)
trading_client = TradingClient(KEY, SECRET_KEY, paper=True)

### Check positions

In [4]:
positions = trading_client.get_all_positions()
sers = [pd.Series(dict(x)) for x in positions]
df = pd.concat(sers, axis=1)
df.T


,asset_id,symbol,exchange,asset_class,avg_entry_price,qty,side,market_value,cost_basis,unrealized_pl,unrealized_plpc,unrealized_intraday_pl,unrealized_intraday_plpc,current_price,lastday_price,change_today
0,b0b6dd9d-8b9b-48a9-ba46-b9d54906e415,AAPL,AssetExchange.NASDAQ,AssetClass.US_EQUITY,135.52,30,PositionSide.LONG,4044,4065.6,-21.6,-0.0053128689492326,-30,-0.0073637702503682,134.8,135.94,-0.0083860526702957
1,8ccae427-5dd0-45b3-b5fe-7ba5e422c766,TSLA,AssetExchange.NASDAQ,AssetClass.US_EQUITY,130,-15,PositionSide.SHORT,-1918.2,-1950,31.8,0.0163076923076923,46.7,0.0237671128301695,127.88,131.49,-0.0274545592820747


### Get quotes for tickers

In [5]:
df = pd.read_csv("files/first_portfolio.csv")
ticks = df.ticker.to_list()

params = StockLatestQuoteRequest(symbol_or_symbols=ticks)
quotes = data_client.get_stock_latest_quote(params)

sers = [pd.Series(dict(x)) for x in quotes.values()]
df = pd.concat(sers, axis=1)
df.columns = [x for x in quotes]
df = df.T
df = df.drop(columns=["symbol"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 126 entries, ORCC to AMCX
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   timestamp     126 non-null    datetime64[ns, UTC]
 1   ask_exchange  126 non-null    object             
 2   ask_price     126 non-null    object             
 3   ask_size      126 non-null    object             
 4   bid_exchange  126 non-null    object             
 5   bid_price     126 non-null    object             
 6   bid_size      126 non-null    object             
 7   conditions    126 non-null    object             
 8   tape          126 non-null    object             
dtypes: datetime64[ns, UTC](1), object(8)
memory usage: 9.8+ KB


### Compute desired trades

In [8]:
equity = 100000
numstocks = len(ticks)
dollars_per_stock = equity / numstocks

df = df[(~df.ask_price.isna()) & (df.ask_price != 0)]
df["trade"] = (dollars_per_stock / df.ask_price).astype(int)
ticks = df[df.trade>0].index.to_list()

### Submit trades

In [9]:
for tick in ticks: 
    market_order_data = MarketOrderRequest(
        symbol=tick,
        qty=df.trade.loc[tick],
        side=OrderSide.BUY,
        time_in_force=TimeInForce.DAY
    )
    market_order = trading_client.submit_order(
        order_data=market_order_data
    )

### Check positions again

In [10]:
positions = trading_client.get_all_positions()
sers = [pd.Series(dict(x)) for x in positions]
df = pd.concat(sers, axis=1)
df.T


,asset_id,symbol,exchange,asset_class,avg_entry_price,qty,side,market_value,cost_basis,unrealized_pl,unrealized_plpc,unrealized_intraday_pl,unrealized_intraday_plpc,current_price,lastday_price,change_today
0,b0b6dd9d-8b9b-48a9-ba46-b9d54906e415,AAPL,AssetExchange.NASDAQ,AssetClass.US_EQUITY,135.52,30,PositionSide.LONG,4043.7,4065.6,-21.9,-0.0053866587957497,-30.3,-0.0074374079528719,134.79,135.94,-0.0084596145358246
1,8ccae427-5dd0-45b3-b5fe-7ba5e422c766,TSLA,AssetExchange.NASDAQ,AssetClass.US_EQUITY,130,-15,PositionSide.SHORT,-1915.05,-1950,34.95,0.0179230769230769,49.85,0.0253702478497633,127.67,131.49,-0.0290516389079017
